In [1]:
from operator import itemgetter
import os
import re
import csv

from typing import List

import numpy as np
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [2]:
# pipeline_dir = '/home/ptoroisaza/fair-fairytale-nlp/data/pipeline/'
pipeline_dir = '/Users/pti/challenges/4761_fair_fairytale/fair-fairytale-nlp/data/pipeline/'

In [3]:
story_id = 'bamboo-cutter-moon-child'

In [4]:
story_dir_with_prefix = pipeline_dir + story_id + '/' + story_id
characters_temporal_df = pd.read_csv(story_dir_with_prefix + '.characters_temporal_events.csv')

In [5]:
characters_df = pd.read_csv(story_dir_with_prefix + '.character_attributes.csv').rename(columns = {'coref_idx': 'coref_id'})

In [6]:
sentences_df = pd.read_csv(story_dir_with_prefix + '.sentences.csv')

In [7]:
characters_temporal_df

,sentence_id,event_id,event,verb_start_byte,verb_end_byte,verb_start_byte_text,verb_end_byte_text,coref_id,arg_start_byte_sentence,arg_end_byte_sentence,...,supersense_category,temporal_rank,argument,name,name_mentions,pronoun_mentions,total_mentions,gender,gender_certainty,importance
0,1,2,sent,55,59,110,114,10,48.0,54.0,...,verb.communication,0,subject,Heaven,1.0,0.0,1.0,unknown,0.0,tertiary
1,1,3,cheer,63,68,118,123,10,48.0,54.0,...,verb.emotion,1,subject,Heaven,1.0,0.0,1.0,unknown,0.0,tertiary
2,1,5,died,143,147,198,202,17,140.0,142.0,...,verb.change,2,subject,an old bamboo wood - cutter,1.0,15.0,16.0,male,1.0,tertiary
3,1,7,laid,156,160,211,215,17,140.0,142.0,...,verb.contact,3,subject,an old bamboo wood - cutter,1.0,15.0,16.0,male,1.0,tertiary
4,3,1,made,12,16,368,372,17,5.0,7.0,...,verb.social,4,subject,an old bamboo wood - cutter,1.0,15.0,16.0,male,1.0,tertiary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,236,1,giving,65,71,29469,29475,29,72.0,76.0,...,verb.possession,584,direct_object,the old man,24.0,81.0,105.0,male,1.0,primary
1078,236,1,giving,65,71,29469,29475,62,72.0,76.0,...,verb.possession,584,direct_object,the Princess,36.0,118.0,154.0,female,1.0,primary
1079,236,2,asked,97,102,29501,29506,231,103.0,106.0,...,verb.communication,585,direct_object,The Emperor,13.0,39.0,52.0,male,1.0,secondary
1080,236,2,asked,97,102,29501,29506,29,103.0,106.0,...,verb.communication,585,direct_object,the old man,24.0,81.0,105.0,male,1.0,primary


In [8]:
lemmatizer = WordNetLemmatizer()

In [9]:
characters_temporal_df['event_lemma'] = characters_temporal_df['event'].apply(lemmatizer.lemmatize, args = ['v'])

## By Character

### Event Counts

In [10]:
char_event_counts = pd.DataFrame(characters_temporal_df['coref_id'].value_counts()).reset_index().rename(columns = {'coref_id': 'event_n', 'index': 'coref_id'})

In [11]:
char_event_counts

,coref_id,event_n
0,62,190
1,29,127
2,11,124
3,137,92
4,231,69
...,...,...
124,187,1
125,262,1
126,403,1
127,103,1


### Argument Counts

In [12]:
char_arg_counts = pd.DataFrame(characters_temporal_df.groupby('coref_id')['argument'].value_counts()).rename(columns = {'argument': 'arg_n'}).reset_index()

In [13]:
char_arg_counts

,coref_id,argument,arg_n
0,10,subject,2
1,11,direct_object,65
2,11,subject,59
3,12,direct_object,2
4,12,subject,2
...,...,...,...
176,389,subject,1
177,402,subject,2
178,403,subject,1
179,404,subject,1


In [14]:
char_arg_counts = pd.pivot(char_arg_counts, index = ['coref_id'], columns = ['argument'], values = ['arg_n'])

In [15]:
char_arg_counts.columns = char_arg_counts.columns.to_flat_index()

In [16]:
char_arg_counts = char_arg_counts.rename(columns = {('arg_n', 'subject'): 'subject_n', ('arg_n', 'direct_object'): 'direct_object_n'}).reset_index().fillna(0)

In [17]:
char_arg_counts

,coref_id,direct_object_n,subject_n
0,10,0.0,2.0
1,11,65.0,59.0
2,12,2.0,2.0
3,13,4.0,0.0
4,14,0.0,1.0
...,...,...,...
124,389,1.0,1.0
125,402,0.0,2.0
126,403,0.0,1.0
127,404,0.0,1.0


### Supersense Counts

In [18]:
char_supersense_counts = pd.DataFrame(characters_temporal_df.groupby('coref_id')['supersense_category'].value_counts()).rename(columns = {'supersense_category': 'supersense_total_n'}).reset_index()

In [19]:
char_supersense_counts

,coref_id,supersense_category,supersense_total_n
0,10,verb.communication,1
1,10,verb.emotion,1
2,11,verb.communication,38
3,11,verb.motion,17
4,11,verb.social,15
...,...,...,...
345,402,verb.communication,1
346,402,verb.social,1
347,403,verb.communication,1
348,404,verb.communication,1


In [20]:
char_supersense_counts = pd.pivot(char_supersense_counts, index = ['coref_id'], columns = ['supersense_category'], values = ['supersense_total_n'])

In [21]:
char_supersense_counts.columns = char_supersense_counts.columns.to_flat_index()

In [22]:
supersense_rename = {('supersense_total_n', 'verb.body'): 'supersense_body_total_n',
                     ('supersense_total_n', 'verb.change'): 'supersense_change_total_n',
                     ('supersense_total_n', 'verb.cognition'): 'supersense_cognition_total_n',
                     ('supersense_total_n', 'verb.communication'): 'supersense_communication_total_n',
                     ('supersense_total_n', 'verb.competition'): 'supersense_competition_total_n',
                     ('supersense_total_n', 'verb.consumption'): 'supersense_consumption_total_n',
                     ('supersense_total_n', 'verb.contact'): 'supersense_contact_total_n',
                     ('supersense_total_n', 'verb.creation'): 'supersense_creation_total_n',
                     ('supersense_total_n', 'verb.emotion'): 'supersense_emotion_total_n',
                     ('supersense_total_n', 'verb.motion'): 'supersense_motion_total_n',
                     ('supersense_total_n', 'verb.perception'): 'supersense_perception_total_n',
                     ('supersense_total_n', 'verb.possession'): 'supersense_possession_total_n',
                     ('supersense_total_n', 'verb.social'): 'supersense_social_total_n'                    
                    }

In [23]:
char_supersense_counts = char_supersense_counts.rename(columns = supersense_rename).reset_index().fillna(0)

In [24]:
char_supersense_counts

,coref_id,supersense_body_total_n,supersense_change_total_n,supersense_cognition_total_n,supersense_communication_total_n,supersense_competition_total_n,supersense_consumption_total_n,supersense_contact_total_n,supersense_creation_total_n,supersense_emotion_total_n,supersense_motion_total_n,supersense_perception_total_n,supersense_possession_total_n,supersense_social_total_n
0,10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,11,0.0,7.0,14.0,38.0,0.0,0.0,9.0,0.0,4.0,17.0,13.0,7.0,15.0
2,12,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,13,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,14,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,389,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
125,402,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
126,403,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,404,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Supersense Argument Counts

In [25]:
char_supersense_arg_counts = pd.DataFrame(characters_temporal_df.groupby(['coref_id', 'argument'])['supersense_category'].value_counts()).rename(columns = {'supersense_category': 'supersense_total_n'}).reset_index()

In [26]:
char_supersense_arg_counts

,coref_id,argument,supersense_category,supersense_total_n
0,10,subject,verb.communication,1
1,10,subject,verb.emotion,1
2,11,direct_object,verb.communication,23
3,11,direct_object,verb.motion,10
4,11,direct_object,verb.perception,8
...,...,...,...,...
456,402,subject,verb.communication,1
457,402,subject,verb.social,1
458,403,subject,verb.communication,1
459,404,subject,verb.communication,1


In [27]:
char_supersense_arg_counts = pd.pivot(data = char_supersense_arg_counts, index = ['coref_id'], columns = ['argument', 'supersense_category'], values = ['supersense_total_n'])

In [28]:
char_supersense_arg_counts.columns = char_supersense_arg_counts.columns.to_flat_index()

In [29]:
char_supersense_arg_counts.columns

Index([      ('supersense_total_n', 'subject', 'verb.communication'),
                   ('supersense_total_n', 'subject', 'verb.emotion'),
       ('supersense_total_n', 'direct_object', 'verb.communication'),
              ('supersense_total_n', 'direct_object', 'verb.motion'),
          ('supersense_total_n', 'direct_object', 'verb.perception'),
              ('supersense_total_n', 'direct_object', 'verb.social'),
          ('supersense_total_n', 'direct_object', 'verb.possession'),
              ('supersense_total_n', 'direct_object', 'verb.change'),
           ('supersense_total_n', 'direct_object', 'verb.cognition'),
             ('supersense_total_n', 'direct_object', 'verb.contact'),
             ('supersense_total_n', 'direct_object', 'verb.emotion'),
                 ('supersense_total_n', 'subject', 'verb.cognition'),
                    ('supersense_total_n', 'subject', 'verb.social'),
                    ('supersense_total_n', 'subject', 'verb.motion'),
                   (

In [30]:
supersense_arg_rename = {('supersense_total_n', 'subject', 'verb.communication'): 'supersense_communication_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.communication'): 'supersense_communication_dobj_n',
                         ('supersense_total_n', 'direct_object', 'verb.motion'): 'supersense_motion_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.cognition'): 'supersense_cognition_subj_n',
                         ('supersense_total_n', 'subject', 'verb.social'): 'supersense_social_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.perception'): 'supersense_perception_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.motion'): 'supersense_motion_subj_n',
                         ('supersense_total_n', 'subject', 'verb.contact'): 'supersense_contact_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.social'): 'supersense_social_dobj_n',
                         ('supersense_total_n', 'direct_object', 'verb.possession'): 'supersense_possession_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.perception'): 'supersense_perception_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.cognition'): 'supersense_cognition_dobj_n',
                         ('supersense_total_n', 'direct_object', 'verb.change'): 'supersense_change_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.change'): 'supersense_change_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.contact'): 'supersense_contact_dobj_n',
                         ('supersense_total_n', 'direct_object', 'verb.emotion'): 'supersense_emotion_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.possession'): 'supersense_possession_subj_n',
                         ('supersense_total_n', 'subject', 'verb.creation'): 'supersense_creation_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.body'): 'supersense_body_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.body'): 'supersense_body_subj_n',
                         ('supersense_total_n', 'subject', 'verb.competition'): 'supersense_competition_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.competition'): 'supersense_competition_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.consumption'): 'supersense_consumption_subj_n',
                         ('supersense_total_n', 'direct_object', 'verb.consumption'): 'supersense_consumption_dobj_n',
                         ('supersense_total_n', 'direct_object', 'verb.creation'): 'supersense_creation_dobj_n',
                         ('supersense_total_n', 'subject', 'verb.emotion'): 'supersense_emotion_subj_n'
                        }

In [31]:
char_supersense_arg_counts = char_supersense_arg_counts.rename(columns = supersense_arg_rename).reset_index().fillna(0)

In [32]:
char_supersense_arg_counts

,coref_id,supersense_communication_subj_n,supersense_emotion_subj_n,supersense_communication_dobj_n,supersense_motion_dobj_n,supersense_perception_dobj_n,supersense_social_dobj_n,supersense_possession_dobj_n,supersense_change_dobj_n,supersense_cognition_dobj_n,...,supersense_change_subj_n,supersense_possession_subj_n,supersense_body_dobj_n,supersense_creation_subj_n,supersense_body_subj_n,supersense_competition_subj_n,supersense_competition_dobj_n,supersense_consumption_dobj_n,supersense_consumption_subj_n,supersense_creation_dobj_n
0,10,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,11,15.0,2.0,23.0,10.0,8.0,6.0,5.0,4.0,4.0,...,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,12,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,389,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,402,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,403,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,404,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
characters_df

,clustered_names,coref_id,name_mentions,pronoun_mentions,total,easy_name,gender,gender_certainty,importance
0,the Princess/The Princess,62,36,118,154,the Princess,female,1.0,primary
1,the old man/The old man,29,24,81,105,the old man,male,1.0,primary
2,the Princess Moonlight/Princess Moonlight,11,20,79,99,the Princess Moonlight,female,1.0,secondary
3,The Knight/the Knight,137,10,54,64,The Knight,male,1.0,secondary
4,The Emperor/the Emperor,231,13,39,52,The Emperor,male,1.0,secondary
...,...,...,...,...,...,...,...,...,...
159,an old man,375,1,0,1,an old man,male,1.0,tertiary
160,the Princess Moonlight/Princess Moonlight--her...,103,1,0,1,the Princess Moonlight's own father,female,0.5,tertiary
161,the old man/The old man--my own child,374,1,0,1,the old man's own child,unknown,0.0,tertiary
162,a heavenly being,373,1,0,1,a heavenly being,unknown,0.0,tertiary


In [34]:
characters_df.merge(char_event_counts, on = 'coref_id', how = 'left')

,clustered_names,coref_id,name_mentions,pronoun_mentions,total,easy_name,gender,gender_certainty,importance,event_n
0,the Princess/The Princess,62,36,118,154,the Princess,female,1.0,primary,190.0
1,the old man/The old man,29,24,81,105,the old man,male,1.0,primary,127.0
2,the Princess Moonlight/Princess Moonlight,11,20,79,99,the Princess Moonlight,female,1.0,secondary,124.0
3,The Knight/the Knight,137,10,54,64,The Knight,male,1.0,secondary,92.0
4,The Emperor/the Emperor,231,13,39,52,The Emperor,male,1.0,secondary,69.0
...,...,...,...,...,...,...,...,...,...,...
159,an old man,375,1,0,1,an old man,male,1.0,tertiary,NaN
160,the Princess Moonlight/Princess Moonlight--her...,103,1,0,1,the Princess Moonlight's own father,female,0.5,tertiary,1.0
161,the old man/The old man--my own child,374,1,0,1,the old man's own child,unknown,0.0,tertiary,2.0
162,a heavenly being,373,1,0,1,a heavenly being,unknown,0.0,tertiary,2.0


In [35]:
count_dfs = [char_event_counts, char_arg_counts, char_supersense_counts, char_supersense_arg_counts]

In [36]:
for i, df in enumerate(count_dfs):
    if i == 0:
        characters_stats_df = characters_df.merge(df, on = 'coref_id', how = 'left')
    else:
        characters_stats_df = characters_stats_df.merge(df, on = 'coref_id', how = 'left')

In [37]:
characters_stats_df = characters_stats_df.fillna(0)

In [38]:
character_stats_json = characters_stats_df.set_index('coref_id').to_dict('index')

In [39]:
character_stats_json[62]

{'clustered_names': 'the Princess/The Princess',
 'name_mentions': 36,
 'pronoun_mentions': 118,
 'total': 154,
 'easy_name': 'the Princess',
 'gender': 'female',
 'gender_certainty': 1.0,
 'importance': 'primary',
 'event_n': 190.0,
 'direct_object_n': 102.0,
 'subject_n': 88.0,
 'supersense_body_total_n': 2.0,
 'supersense_change_total_n': 11.0,
 'supersense_cognition_total_n': 21.0,
 'supersense_communication_total_n': 59.0,
 'supersense_competition_total_n': 2.0,
 'supersense_consumption_total_n': 3.0,
 'supersense_contact_total_n': 11.0,
 'supersense_creation_total_n': 1.0,
 'supersense_emotion_total_n': 10.0,
 'supersense_motion_total_n': 17.0,
 'supersense_perception_total_n': 17.0,
 'supersense_possession_total_n': 15.0,
 'supersense_social_total_n': 21.0,
 'supersense_communication_subj_n': 22.0,
 'supersense_emotion_subj_n': 6.0,
 'supersense_communication_dobj_n': 37.0,
 'supersense_motion_dobj_n': 7.0,
 'supersense_perception_dobj_n': 7.0,
 'supersense_social_dobj_n': 9.0,


### Individual Event Counts

In [40]:
for coref_id in character_stats_json.keys():
    char_events_df = characters_temporal_df[characters_temporal_df['coref_id'] == coref_id]
    character_stats_json[coref_id]['event_counts_total'] = char_events_df['event_lemma'].value_counts().to_dict()
    character_stats_json[coref_id]['event_counts_subj'] = char_events_df[char_events_df['argument'] == 'subject']['event_lemma'].value_counts().to_dict()
    character_stats_json[coref_id]['event_counts_dobj'] = char_events_df[char_events_df['argument'] == 'direct_object']['event_lemma'].value_counts().to_dict()

## By Event

In [41]:
event_counts = pd.DataFrame(characters_temporal_df['event_lemma'].value_counts()).reset_index().rename(columns = {'event_lemma': 'event_n', 'index': 'event_lemma'})

In [42]:
event_counts

,event_lemma,event_n
0,tell,70
1,say,49
2,see,41
3,send,33
4,give,29
...,...,...
217,enter,1
218,dismount,1
219,revive,1
220,disappear,1


In [43]:
event_gender_counts = pd.crosstab(characters_temporal_df['event_lemma'], characters_temporal_df['gender'])

In [44]:
event_gender_counts

gender,female,group/nonbinary,male,unknown
event_lemma,,,,
accept,0,1,0,0
accompany,2,1,2,0
add,4,1,1,0
age,0,0,1,0
allow,2,3,2,0
...,...,...,...,...
withstand,0,0,1,1
wonder,1,2,2,2
work,0,1,0,0


In [45]:
event_gender_counts = event_gender_counts.reset_index()

In [46]:
event_gender_counts.columns.name = None

In [47]:
rename_columns = {'female': 'female_event_n',
                  'male': 'male_event_n',
                  'group/nonbinary': 'group/nonbinary_event_n',
                  'unknown': 'unknown_event_n'}

In [48]:
event_gender_counts.rename(columns = rename_columns, inplace = True)

In [49]:
event_gender_counts

,event_lemma,female_event_n,group/nonbinary_event_n,male_event_n,unknown_event_n
0,accept,0,1,0,0
1,accompany,2,1,2,0
2,add,4,1,1,0
3,age,0,0,1,0
4,allow,2,3,2,0
...,...,...,...,...,...
217,withstand,0,0,1,1
218,wonder,1,2,2,2
219,work,0,1,0,0
220,wrap,0,0,1,0


### By Argument

In [50]:
event_arg_counts = pd.crosstab(characters_temporal_df['event_lemma'], characters_temporal_df['argument']).reset_index()

In [51]:
# event_arg_gender_counts.drop(event_arg_gender_counts.tail(1).index, inplace = True)

In [52]:
event_arg_counts.columns.name = None

In [53]:
rename_columns = {'direct_object': 'dobj_event_n',
                  'subject': 'subj_event_n'}

In [54]:
event_arg_counts.rename(columns = rename_columns, inplace = True)

In [55]:
event_arg_counts

,event_lemma,dobj_event_n,subj_event_n
0,accept,0,1
1,accompany,2,3
2,add,5,1
3,age,0,1
4,allow,5,2
...,...,...,...
217,withstand,1,1
218,wonder,4,3
219,work,0,1
220,wrap,0,1


### By Argument and Gender

In [56]:
event_arg_gender_counts = pd.crosstab(characters_temporal_df['event_lemma'], [characters_temporal_df['argument'], characters_temporal_df['gender']])

In [57]:
event_arg_gender_counts.columns = event_arg_gender_counts.columns.to_flat_index()

In [58]:
event_arg_gender_counts

,"(direct_object, female)","(direct_object, group/nonbinary)","(direct_object, male)","(direct_object, unknown)","(subject, female)","(subject, group/nonbinary)","(subject, male)","(subject, unknown)"
event_lemma,,,,,,,,
accept,0,0,0,0,0,1,0,0
accompany,1,0,1,0,1,1,1,0
add,3,1,1,0,1,0,0,0
age,0,0,0,0,0,0,1,0
allow,2,2,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...
withstand,0,0,0,1,0,0,1,0
wonder,0,1,1,2,1,1,1,0
work,0,0,0,0,0,1,0,0


In [59]:
event_arg_gender_counts.columns

Index([         ('direct_object', 'female'),
       ('direct_object', 'group/nonbinary'),
                  ('direct_object', 'male'),
               ('direct_object', 'unknown'),
                      ('subject', 'female'),
             ('subject', 'group/nonbinary'),
                        ('subject', 'male'),
                     ('subject', 'unknown')],
      dtype='object')

In [60]:
rename_columns = {('direct_object', 'female'): 'dobj_female_event_n',
                  ('direct_object', 'group/nonbinary'): 'dobj_group_nb_event_n',
                  ('direct_object', 'male'): 'dobj_male_event_n',
                  ('direct_object', 'unknown'): 'dobj_unknown_event_n',
                  ('subject', 'female'): 'subj_female_event_n',
                  ('subject', 'group/nonbinary'): 'subj_group_nb_event_n',
                  ('subject', 'male'): 'subj_male_event_n',
                  ('subject', 'unknown'): 'subj_unknown_event_n'
                 }

In [61]:
event_arg_gender_counts.rename(columns = rename_columns, inplace = True)

In [62]:
event_arg_gender_counts.reset_index(inplace = True)

### Odds Ratio

In [63]:
def odds_ratio(f_dict, m_dict, topk=50, threshold=2):
    very_small_value = 0.00001
    haldane_correction = 0.5
    if len(f_dict.keys()) != len(m_dict.keys()):
        raise Exception("The category for analyzing the male and female should be the same!")
    else:
        odds_ratio = {}
        total_num_f = sum(f_dict.values())
        total_num_m = sum(m_dict.values())
        for key in f_dict.keys():
            m_num = m_dict[key] + haldane_correction
            f_num = f_dict[key] + haldane_correction
            non_f_num = total_num_f - f_num
            non_m_num = total_num_m - m_num
            # if f_num >= threshold and m_num >= threshold:
                # we only consider the events where there are at least {thresohld} occurences for both gender
            if f_num >= threshold or m_num >= threshold:
                odds_ratio[key] = round((m_num / f_num) / (non_m_num / non_f_num), 2)
            else:
                continue
        return dict(sorted(odds_ratio.items(), key=itemgetter(1), reverse=True))

In [64]:
event_female_dict_raw = event_gender_counts[['event_lemma', 'female_event_n']].to_dict('records')

In [65]:
event_female_dict_raw = event_gender_counts[['event_lemma', 'female_event_n']].to_dict('records')
event_female_dict = {}
for row in event_female_dict_raw:
    event_female_dict[row['event_lemma']] = row['female_event_n']

In [66]:
event_male_dict_raw = event_gender_counts[['event_lemma', 'male_event_n']].to_dict('records')
event_male_dict = {}
for row in event_male_dict_raw:
    event_male_dict[row['event_lemma']] = row['male_event_n']

In [67]:
gender_odds = pd.DataFrame.from_dict(odds_ratio(event_female_dict, event_male_dict), orient = 'index').reset_index().rename(columns = {'index': 'event_lemma', 0: 'male_female_odds'})

In [68]:
event_female_subj_dict_raw = event_arg_gender_counts[['event_lemma', 'subj_female_event_n']].to_dict('records')
event_female_subj_dict = {}
for row in event_female_subj_dict_raw:
    event_female_subj_dict[row['event_lemma']] = row['subj_female_event_n']

In [69]:
event_male_subj_dict_raw = event_arg_gender_counts[['event_lemma', 'subj_male_event_n']].to_dict('records')
event_male_subj_dict = {}
for row in event_male_subj_dict_raw:
    event_male_subj_dict[row['event_lemma']] = row['subj_male_event_n']

In [70]:
subj_gender_odds = pd.DataFrame.from_dict(odds_ratio(event_female_subj_dict, event_male_subj_dict), orient = 'index').reset_index().rename(columns = {'index': 'event_lemma', 0: 'male_female_subj_odds'})

In [71]:
subj_gender_odds

,event_lemma,male_female_subj_odds
0,find,20.86
1,send,14.87
2,think,10.50
3,bring,7.64
4,beg,4.82
...,...,...
58,sit,0.13
59,hide,0.10
60,know,0.10
61,speak,0.10


In [72]:
event_female_dobj_dict_raw = event_arg_gender_counts[['event_lemma', 'dobj_female_event_n']].to_dict('records')
event_female_dobj_dict = {}
for row in event_female_dobj_dict_raw:
    event_female_dobj_dict[row['event_lemma']] = row['dobj_female_event_n']
event_male_dobj_dict_raw = event_arg_gender_counts[['event_lemma', 'dobj_male_event_n']].to_dict('records')
event_male_dobj_dict = {}
for row in event_male_dobj_dict_raw:
    event_male_dobj_dict[row['event_lemma']] = row['dobj_male_event_n']
dobj_gender_odds = pd.DataFrame.from_dict(odds_ratio(event_female_dobj_dict, event_male_dobj_dict), orient = 'index').reset_index().rename(columns = {'index': 'event_lemma', 0: 'male_female_dobj_odds'})

### By Importance

In [73]:
event_importance_counts = pd.crosstab(characters_temporal_df['event_lemma'], characters_temporal_df['importance']).reset_index()

In [74]:
event_importance_counts

importance,event_lemma,primary,secondary,tertiary
0,accept,0,0,1
1,accompany,1,2,2
2,add,3,0,3
3,age,1,0,0
4,allow,1,2,4
...,...,...,...,...
217,withstand,1,0,1
218,wonder,0,3,4
219,work,0,0,1
220,wrap,1,0,0


In [75]:
event_importance_counts.columns = event_importance_counts.columns.to_flat_index()
event_importance_counts.columns.name = None
# rename_columns = {'female': 'female_event_n',
#                   'male': 'male_event_n',
#                   'group/nonbinary': 'group/nonbinary_event_n',
#                   'unknown': 'unknown_event_n'}
# event_importance_counts.rename(columns = rename_columns, inplace = True)

In [76]:
event_importance_counts

,event_lemma,primary,secondary,tertiary
0,accept,0,0,1
1,accompany,1,2,2
2,add,3,0,3
3,age,1,0,0
4,allow,1,2,4
...,...,...,...,...
217,withstand,1,0,1
218,wonder,0,3,4
219,work,0,0,1
220,wrap,1,0,0


In [77]:
rename_columns = {'primary': 'primary_event_n',
                  'secondary': 'secondary_event_n',
                  'tertiary': 'tertiary_event_n'}
event_importance_counts.rename(columns = rename_columns, inplace = True)

In [78]:
event_primary_dict_raw = event_importance_counts[['event_lemma', 'primary_event_n']].to_dict('records')
event_primary_dict = {}
for row in event_primary_dict_raw:
    event_primary_dict[row['event_lemma']] = row['primary_event_n']
    
event_secondary_dict_raw = event_importance_counts[['event_lemma', 'secondary_event_n']].to_dict('records')
event_secondary_dict = {}
for row in event_secondary_dict_raw:
    event_secondary_dict[row['event_lemma']] = row['secondary_event_n']
    
event_tertiary_dict_raw = event_importance_counts[['event_lemma', 'tertiary_event_n']].to_dict('records')
event_tertiary_dict = {}
for row in event_tertiary_dict_raw:
    event_tertiary_dict[row['event_lemma']] = row['tertiary_event_n']

In [79]:
importance_pri_sec_odds = pd.DataFrame.from_dict(odds_ratio(event_secondary_dict, event_primary_dict), orient = 'index').reset_index().rename(columns = {'index': 'event_lemma', 0: 'primary_secondary_odds'})

In [80]:
importance_pri_ter_odds = pd.DataFrame.from_dict(odds_ratio(event_tertiary_dict, event_primary_dict), orient = 'index').reset_index().rename(columns = {'index': 'event_lemma', 0: 'primary_tertiary_odds'})

### Merge Together

In [81]:
count_dfs = [event_counts, event_gender_counts, event_arg_counts, event_arg_gender_counts, gender_odds, subj_gender_odds, dobj_gender_odds, event_importance_counts, importance_pri_sec_odds, importance_pri_ter_odds]

In [82]:
events_df = characters_temporal_df[['event_lemma', 'supersense_category']].drop_duplicates('event_lemma')

In [83]:
for i, df in enumerate(count_dfs):
    if i == 0:
        events_stats_df = events_df.merge(df, on = 'event_lemma', how = 'left')
    else:
        events_stats_df = events_stats_df.merge(df, on = 'event_lemma', how = 'left')

In [84]:
events_stats_df

,event_lemma,supersense_category,event_n,female_event_n,group/nonbinary_event_n,male_event_n,unknown_event_n,dobj_event_n,subj_event_n,dobj_female_event_n,...,subj_male_event_n,subj_unknown_event_n,male_female_odds,male_female_subj_odds,male_female_dobj_odds,primary_event_n,secondary_event_n,tertiary_event_n,primary_secondary_odds,primary_tertiary_odds
0,send,verb.communication,33,6,2,17,8,18,15,6,...,10,4,2.24,14.87,1.15,7,12,14,0.53,0.78
1,cheer,verb.emotion,2,0,0,1,1,1,1,0,...,0,1,NaN,NaN,NaN,0,0,2,NaN,0.30
2,die,verb.change,2,0,0,2,0,0,2,0,...,2,0,4.09,3.43,NaN,1,0,1,NaN,NaN
3,lay,verb.contact,1,0,0,1,0,0,1,0,...,1,0,NaN,NaN,NaN,0,0,1,NaN,NaN
4,make,verb.social,26,11,3,10,2,14,12,7,...,6,0,0.74,0.98,0.58,7,6,13,1.04,0.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,press,verb.social,1,1,0,0,0,1,0,1,...,0,0,NaN,NaN,NaN,0,1,0,NaN,NaN
218,happen,verb.change,2,0,1,0,1,2,0,0,...,0,0,NaN,NaN,NaN,0,0,2,NaN,0.30
219,station,verb.social,1,0,0,0,1,1,0,0,...,0,0,NaN,NaN,NaN,0,0,1,NaN,NaN
220,protect,verb.competition,1,1,0,0,0,1,0,1,...,0,0,NaN,NaN,NaN,1,0,0,NaN,NaN


In [85]:
event_stats_json = events_stats_df.set_index('event_lemma').to_dict('index')

In [86]:
for event_lemma in event_stats_json.keys():
    event_stats_json[event_lemma]['event_occurances'] = {}
    event_lemma_df = characters_temporal_df[characters_temporal_df['event_lemma'] == event_lemma][['sentence_id', 'event_id', 'temporal_rank', 'coref_id', 'argument', 'gender', 'importance']]
    event_ids = pd.Series([tuple(r) for r in event_lemma_df[['sentence_id', 'event_id']].values]).unique()
    for sentence_id, event_id in event_ids:
        story_position = round(sentence_id/len(sentences_df), 2)
        event_i_df = event_lemma_df[(event_lemma_df['sentence_id'] == sentence_id) & (event_lemma_df['event_id'] == event_id)][['temporal_rank', 'coref_id', 'argument', 'gender', 'importance']]
        event_stats_json[event_lemma]['event_occurances'][(sentence_id, event_id, story_position)] = event_i_df.to_dict('records')

## By Story

In [87]:
story_stats_json = {}

### Characters

In [88]:
story_stats_json['characters'] = {}

In [89]:
story_stats_json['characters']['n'] = len(characters_df)

In [90]:
story_stats_json['characters']['n']

164

In [91]:
for gender in ['female', 'male', 'group/nonbinary', 'unknown']:
    story_stats_json['characters'][gender] = {}
    story_stats_json['characters'][gender]['n'] = characters_df['gender'].value_counts()[gender]
    story_stats_json['characters'][gender]['p'] = round(characters_df['gender'].value_counts()[gender]/len(characters_df), 2)

In [92]:
for importance in ['primary', 'secondary', 'tertiary']:
    story_stats_json['characters'][importance] = {}
    story_stats_json['characters'][importance]['n'] = characters_df['importance'].value_counts()[importance]
    story_stats_json['characters'][importance]['p'] = round(characters_df['importance'].value_counts()[importance]/len(characters_df), 2)

In [93]:
char_gender_imp_counts = pd.crosstab(characters_df['gender'], characters_df['importance']).reset_index()

In [94]:
char_gender_imp_counts.columns.name = None

In [95]:
char_gender_imp_counts

,gender,primary,secondary,tertiary
0,female,1,1,26
1,group/nonbinary,0,0,21
2,male,1,2,33
3,unknown,0,0,79


In [96]:
char_gender_imp_p = pd.crosstab(characters_df['gender'], characters_df['importance'], normalize = True).reset_index()

In [97]:
char_gender_imp_p.columns.name = None

In [98]:
for gender in ['female', 'male', 'group/nonbinary', 'unknown']:
    for importance in ['primary', 'secondary', 'tertiary']:
        gender_importance = gender + '_' + importance
        story_stats_json['characters'][gender_importance] = {}
        story_stats_json['characters'][gender_importance]['n'] = char_gender_imp_counts[char_gender_imp_counts['gender'] == gender][importance].values[0]
        story_stats_json['characters'][gender_importance]['p'] = round(char_gender_imp_p[char_gender_imp_p['gender'] == gender][importance].values[0], 2)

In [99]:
story_stats_json['characters']['appearances'] = {}
story_stats_json['characters']['appearances']['all'] = characters_df['total'].tolist()
for gender in ['female', 'male', 'group/nonbinary', 'unknown']:
    story_stats_json['characters']['appearances'][gender] = characters_df[characters_df['gender'] == gender]['total'].tolist()

### Events

In [100]:
story_stats_json['events'] = {}

#### Counts

In [101]:
story_event_counts = pd.crosstab([characters_temporal_df['gender'], characters_temporal_df['importance']], characters_temporal_df['argument'], margins = True, margins_name = 'total').reset_index()

In [102]:
story_event_gender_counts = pd.crosstab(characters_temporal_df['gender'], characters_temporal_df['argument'], margins = True, margins_name = 'total').reset_index()

In [103]:
story_event_counts.columns.name = None

In [104]:
story_event_gender_counts.drop(story_event_gender_counts.tail(1).index, inplace = True)

In [105]:
story_event_gender_counts['importance'] = 'total'

In [106]:
story_event_gender_counts.columns.name = None

In [107]:
story_event_counts = pd.concat([story_event_counts, story_event_gender_counts]).sort_values(by = 'gender')

In [108]:
story_event_counts

,gender,importance,direct_object,subject,total
0,female,primary,102,88,190
1,female,secondary,65,59,124
2,female,tertiary,29,33,62
0,female,total,196,180,376
3,group/nonbinary,tertiary,68,67,135
1,group/nonbinary,total,68,67,135
4,male,primary,53,74,127
5,male,secondary,67,94,161
6,male,tertiary,78,96,174
2,male,total,198,264,462


In [ ]:
story_stats_json['events']['counts'] = story_event_counts.to_dict('records')

#### Top Events

In [109]:
story_stats_json['events']['top_events'] = {}

In [110]:
subj_gender_odds['argument'] = 'subject'
dobj_gender_odds['argument'] = 'direct_object'
subj_gender_odds.rename(columns = {'male_female_subj_odds': 'male_female_odds'}, inplace = True)
dobj_gender_odds.rename(columns = {'male_female_dobj_odds': 'male_female_odds'}, inplace = True)

In [111]:
subj_gender_odds

,event_lemma,male_female_odds,argument
0,find,20.86,subject
1,send,14.87,subject
2,think,10.50,subject
3,bring,7.64,subject
4,beg,4.82,subject
...,...,...,...
58,sit,0.13,subject
59,hide,0.10,subject
60,know,0.10,subject
61,speak,0.10,subject


In [112]:
gender_odds_top_male = pd.concat([subj_gender_odds, dobj_gender_odds]).sort_values(by = 'male_female_odds', ascending = False)

In [113]:
gender_odds_top_male

,event_lemma,male_female_odds,argument
0,find,20.86,subject
1,send,14.87,subject
2,think,10.50,subject
3,bring,7.64,subject
0,ask,5.82,direct_object
...,...,...,...
61,speak,0.10,subject
60,know,0.10,subject
59,hide,0.10,subject
62,consent,0.07,subject


In [114]:
gender_odds_top_female = gender_odds_top_male.copy()

In [115]:
gender_odds_top_female['female_male_odds'] = round(1/gender_odds_top_female['male_female_odds'], 2)

In [116]:
gender_odds_top_female.drop(columns = 'male_female_odds', inplace = True)

In [117]:
gender_odds_top_female.sort_values(by = 'female_male_odds', ascending = False, inplace = True)

In [118]:
gender_odds_top_male = gender_odds_top_male[gender_odds_top_male['male_female_odds'] >= 2].to_dict('records')

In [119]:
gender_odds_top_female = gender_odds_top_female[gender_odds_top_female['female_male_odds'] >= 2].to_dict('records')

In [120]:
story_stats_json['events']['top_events'] = {}
story_stats_json['events']['top_events']['female'] = gender_odds_top_female
story_stats_json['events']['top_events']['male'] = gender_odds_top_male

# for event in gender_odds_top_female:
#     event_lemma = event['event_lemma']
#     story_stats_json['events']['top_events']['female'][event_lemma] = {'argument': event['argument'], 'odds': event['female_male_odds']}

In [121]:
story_stats_json['events']['top_events']['female']

[{'event_lemma': 'win',
  'argument': 'direct_object',
  'female_male_odds': 16.67},
 {'event_lemma': 'consent', 'argument': 'subject', 'female_male_odds': 14.29},
 {'event_lemma': 'hide', 'argument': 'subject', 'female_male_odds': 10.0},
 {'event_lemma': 'know', 'argument': 'subject', 'female_male_odds': 10.0},
 {'event_lemma': 'speak', 'argument': 'subject', 'female_male_odds': 10.0},
 {'event_lemma': 'desire', 'argument': 'subject', 'female_male_odds': 7.69},
 {'event_lemma': 'do', 'argument': 'subject', 'female_male_odds': 7.69},
 {'event_lemma': 'force', 'argument': 'subject', 'female_male_odds': 7.69},
 {'event_lemma': 'forget', 'argument': 'subject', 'female_male_odds': 7.69},
 {'event_lemma': 'sit', 'argument': 'subject', 'female_male_odds': 7.69},
 {'event_lemma': 'feel', 'argument': 'subject', 'female_male_odds': 7.69},
 {'event_lemma': 'place',
  'argument': 'direct_object',
  'female_male_odds': 7.14},
 {'event_lemma': 'bring',
  'argument': 'direct_object',
  'female_male_

In [124]:
story_stats_json['events']

{'top_events': {'female': [{'event_lemma': 'win',
    'argument': 'direct_object',
    'female_male_odds': 16.67},
   {'event_lemma': 'consent',
    'argument': 'subject',
    'female_male_odds': 14.29},
   {'event_lemma': 'hide', 'argument': 'subject', 'female_male_odds': 10.0},
   {'event_lemma': 'know', 'argument': 'subject', 'female_male_odds': 10.0},
   {'event_lemma': 'speak', 'argument': 'subject', 'female_male_odds': 10.0},
   {'event_lemma': 'desire', 'argument': 'subject', 'female_male_odds': 7.69},
   {'event_lemma': 'do', 'argument': 'subject', 'female_male_odds': 7.69},
   {'event_lemma': 'force', 'argument': 'subject', 'female_male_odds': 7.69},
   {'event_lemma': 'forget', 'argument': 'subject', 'female_male_odds': 7.69},
   {'event_lemma': 'sit', 'argument': 'subject', 'female_male_odds': 7.69},
   {'event_lemma': 'feel', 'argument': 'subject', 'female_male_odds': 7.69},
   {'event_lemma': 'place',
    'argument': 'direct_object',
    'female_male_odds': 7.14},
   {'eve